# BS Optimization

- Solving a constrained resource allocation problem using a sample Bank balance sheet and (mostly) realistic balance sheet constraints

### Background

Bank resource management has become significantly more complex in the last decade. After the 2008 financial crisis, a slew of new regulatory frameworks were enacted, complicating balance sheet management. It was no longer immediately obvious which products were more profitable or what the optimal balance sheet mix was for a given "type" of bank (i.e., Universal vs. Investment vs. Retail bank, etc).

This is where optimization can help. These constraints can be expressed as equations, and by creating bounds on the Balance Sheet (abbreviated here on out as B/S) line items, these equations can be solved in a way that maximizes the profitability of the balance sheet.

### Constraint overview

In [348]:
import pandas as pd
import numpy as np
import scipy.optimize as spo

In [349]:
# Set options to print all columns of the dataframe

pd.set_option('display.max_columns', None)

In [352]:
# Read in BS data

df = pd.read_excel('./data/sample_bs.xlsx', index_col=0, nrows=39, usecols='A:AN')

In [353]:
df.head()

,Product,start,grow,shrink,spread,A_L,b1_leverage,a_rwa,s_rwa,gsib_leverage,gsib_xjd_claim,gsib_xjd_liab,gsib_intrafin_claim,gsib_intrafin_liab,gsib_securities,gsib_payment,gsib_auc,gsib_underwriting,gsib_otc,gsib_trading,gsib_level3,CET1_resource,T1_resource,total_capital_resource,TLAC_resource,gsib_leverage_score,gsib_xjd_claim_score,gsib_xjd_liab_score,gsib_intrafin_claim_score,gsib_intrafin_liab_score,gsib_securities_score,gsib_payment_score,gsib_auc_score,gsib_underwriting_score,gsib_otc_score,gsib_trading_score,gsib_level3_score,total_score,cet1_contr
Index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,prime_auto,15000,8000,-5000,80,1,1,0.2,1.0,1,0.0,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,3.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,3.0,0.000144
2,subprime_auto,4000,2000,-1000,150,1,1,0.8,1.0,1,0.0,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.8,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0.8,0.000039
3,mtg_30_fixed,50000,30000,-15000,70,1,1,0.3,0.5,1,0.0,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,10.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,10.0,0.000482
4,mtg_15_fixed,15000,8000,-3000,72,1,1,0.3,0.5,1,0.0,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,3.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,3.0,0.000144
5,mtg_7_fixed,5000,2500,-2000,70,1,1,0.3,0.5,1,0.0,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,1.0,0.000048


In [359]:
# Read in constraints

constraints = pd.read_excel('./data/sample_bs.xlsx', index_col=0, sheet_name='constraints')

constraints

,CET1,T1,total_capital,TLAC
SRWA,0.080000,0.095,0.115,0.195
ARWA,0.080000,0.095,0.115,0.195
leverage,0.075000,0.090,NaN,NaN
GSIB,0.000048,NaN,NaN,NaN


## Objective function

- This is simply an array of the negative return (i.e., profitability)
- Since the optimization will actually be minimizing this, we just flip the sign 

In [357]:
# Define objective function coefficients
# We are 'minimizing' the spread so will make this negative
# Dividing by 10,000 to convert bps to spread (will not affect optimization)

c = -np.array(df['spread'] / 10000)

## Inequality Constraint

- The inequality contraints state that you must have enough resources (of the correct type) to cover a given % of the risk weighted assets on the B/S.
- Each B/S line item contributes some amount to RWA
- Taking CET1 as an example, there must be enough equity to cover 8% of RWA plus the GSIB surcharge
- This GSIB surcharge has beene "linearized" and each B/S line item adds (or subtracts) a certain amount from this minimum requirement as balances change
- The final formula for CET1 is as follows:
    - The inequality constraint for our optimization is the the RWA % * (8% base + GSIB surcharge) * the balance - equity needs to be greater than or equal to 0.
    - For other constraints in the resource stack, additional resources count towards fulfilling this requirement. For example, for T1 capital, this constraint can be met with both Equity and Prefs. However, the underlying principle is hte same.

### Advanced RWA

In [334]:
# Extract Advanced RWA inequality coefficients

aRWA_mins = list(constraints.loc['ARWA', :])

aRWA_cet1 = list(df['a_rwa'] * (aRWA_mins[0] + df['cet1_contr'].sum()) - df['CET1_resource'])
aRWA_t1 = list(df['a_rwa'] * (aRWA_mins[1] + df['cet1_contr'].sum()) - df['T1_resource'])
aRWA_tc = list(df['a_rwa'] * (aRWA_mins[2] + df['cet1_contr'].sum()) - df['total_capital_resource'])
aRWA_tlac = list(df['a_rwa'] * (aRWA_mins[3] + df['cet1_contr'].sum()) - df['TLAC_resource'])

# Combine into one list
aRWA_constraints = [aRWA_cet1, aRWA_t1, aRWA_tc, aRWA_tlac]

### Standardized RWA

In [335]:
# Extract RWA inequality coefficients

sRWA_mins = list(constraints.loc['SRWA', :])

sRWA_cet1 = list(df['s_rwa'] * (sRWA_mins[0] + df['cet1_contr'].sum()) - df['CET1_resource'])
sRWA_t1 = list(df['s_rwa'] * (sRWA_mins[1] + df['cet1_contr'].sum()) - df['T1_resource'])
sRWA_tc = list(df['s_rwa'] * (sRWA_mins[2] + df['cet1_contr'].sum()) - df['total_capital_resource'])
sRWA_tlac = list(df['s_rwa'] * (sRWA_mins[3] + df['cet1_contr'].sum()) - df['TLAC_resource'])

# Combine into one list

sRWA_constraints = [sRWA_cet1, sRWA_t1, sRWA_tc, sRWA_tlac]

### Leverage

- Leverage works very similarly to the RWA constraints with two exceptions:
    - There is no RWA intermediary. The Asset balances are directly multiplied by the CET1 (or T1) minimums.
    - GSIB is not a factor

In [336]:
# Extract leverage inequality constraint

lev_mins = list(constraints.loc['leverage',['CET1', 'T1']])

lev_cet1 = list(df['b1_leverage'] * lev_mins[0] - df['CET1_resource'])
lev_t1 = list(df['b1_leverage'] * lev_mins[1] - df['T1_resource'])

# Combine into one list

lev_constraints = [lev_cet1, lev_t1]

In [337]:
# Combine Advanced RWA and Standardized RWA constraints into a single list

A_ineq = aRWA_constraints + sRWA_constraints + lev_constraints

# Define the other side of the inequality equation (we want to be at least at the minimum, so this is 0)
# There are 10 constraints so this gets multiplied accordingly

b_ineq = [0] * 10

In [338]:
A_ineq

[[0.022067423847172616,
  0.08826969538869046,
  0.03310113577075892,
  0.03310113577075892,
  0.03310113577075892,
  0.03310113577075892,
  0.03310113577075892,
  0.13240454308303568,
  0.10482026327406992,
  0.0827528394268973,
  0.07723598346510414,
  0.0827528394268973,
  0.071719127503311,
  0.04413484769434523,
  0.05516855961793154,
  0.04413484769434523,
  0.05516855961793154,
  0.022067423847172616,
  0.005516855961793154,
  0.11033711923586308,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  -1.0,
  0.0,
  0.0,
  0.0],
 [0.025067423847172615,
  0.10026969538869046,
  0.03760113577075892,
  0.03760113577075892,
  0.03760113577075892,
  0.03760113577075892,
  0.03760113577075892,
  0.1504045430830357,
  0.11907026327406992,
  0.0940028394268973,
  0.08773598346510415,
  0.0940028394268973,
  0.081469127503311,
  0.05013484769434523,
  0.06266855961793154,
  0.05013484769434523,
  0.06266855961793154,
  0.025067423847172

## Equality Constraint

- This constraint states that Assets - Liabilities must equal to 0. This makes sense as our balance sheet always needs to balance
- I've pre-programmed this to an extent by including a assets (+100%) and liability (-100%) weight in the inputs.

In [339]:
# Create Asset and Liability equality constraint.

A_eq = [list(df['A_L'])]

In [340]:
# Equality constraint vector defined below. This is saying that assets must equal liabilities on the B/S

a_l_constraint = 0

b_eq = [a_l_constraint]

In [341]:
# Create upper and lower bounds. This is the grow and shrink amount for each item

df['u_bound'] = df['start'] + df['grow']
df['l_bound'] = df['start'] + df['shrink']

# Create tuple of lower / upper bounds
df['bounds'] = df.apply(lambda row: tuple((row['l_bound'], row['u_bound'])), axis=1)

In [342]:
# Define bounds in format needed for linprog

bounds = list(df['bounds'])

## Optimize B/S

In [343]:
res = spo.linprog(c, A_ub = A_ineq, b_ub = b_ineq, A_eq = A_eq, b_eq = b_eq, bounds = bounds, 
                  method='revised simplex', options = {"disp": True})

Phase Iteration Minimum Slack       Constraint Residual Objective          
1     0         -82819.93907021     128450.0            -13980.8            
1     1         -49814.48761438     95444.54854417      -10680.25485442     
1     2         -46123.57511438     91753.63604417      -10311.16360442     
1     3         -43411.18907021     89041.25            -10039.925          
1     4         -41369.02511438     86999.08604417      -9835.708604417     
1     5         -40339.2875         85969.34842979      -9732.734842979     
1     6         -36229.43907021     81859.5             -9465.594695043     
1     7         -35360.25           80990.31092979      -9409.097405479     
1     8         -28888.2            74518.26092979      -9279.656405479     
1     9         -19350.82511438     64980.88604417      -9165.207906851     
1     10        0.0                 45630.06092979      -8932.998005479     
1     11        0.0                 0.0                 -9845.599224074     


## Results

In [344]:
# This shows that a solution was found

res.success

True

In [345]:
# Show the output of the objective function

res.fun

-15593.702898748703

In [346]:
# Compare to starting profitability

start_profitability = -sum(df['start'] * (df['spread'] / 10000))

(res.fun - start_profitability) / start_profitability

# Profitability increased 50%

0.42894347426163915

In [347]:
# Show the slack against each of the constraints.
# As expect, Standardized RWA (last four values) is significantly more binding than Advanced

res.slack

array([ 3.90979900e+03,  8.55992787e+03, -9.09494702e-12, -2.36468622e-11,
        3.90979900e+03,  8.55992787e+03, -1.09139364e-11,  9.09494702e-12,
        0.00000000e+00, -1.09139364e-11])

In [321]:
# Show ending balances

res.x

array([ 10000.        ,   6000.        ,  80000.        ,  23000.        ,
         7500.        ,  30000.        ,  13500.        ,  80000.        ,
        17500.        ,  35000.        ,  62706.93548387,  28000.        ,
        85000.        , 105000.        ,  75000.        ,  95000.        ,
        34000.        ,  13000.        ,   1500.        , 240000.        ,
        88500.        , 110000.        ,  26000.        ,  30000.        ,
         3500.        ,  75000.        ,  32000.        ,  22000.        ,
        12500.        ,  16000.        ,   9500.        ,   5000.        ,
         2000.        ,   3000.        ,  61945.61290323,  10208.01129032,
         9501.95645161,  45051.35483871])

In [322]:
# Append ending balance to our data and compare growth vs. shrink

df['optimal_balance'] = res.x

df['balance_change'] = df['optimal_balance'] - df['start']

In [323]:
balance_results = df[['Product', 'balance_change']]

In [324]:
# Some products are growing, some are shrinking, and business non-op deposits are our marginal product

balance_results

,Product,balance_change
Index,,
1,prime_auto,-5000.000000
2,subprime_auto,2000.000000
3,mtg_30_fixed,30000.000000
4,mtg_15_fixed,8000.000000
5,mtg_7_fixed,2500.000000
6,mtg_15_arm,10000.000000
7,mtg_7_arm,3500.000000
8,consumer_card,20000.000000
9,business_card,2500.000000


In [176]:
df.head()

,Product,start,grow,shrink,spread,A_L,a_rwa,s_rwa,CET1_resource,T1_resource,total_capital_resource,TLAC_resource,u_bound,l_bound,bounds,optimal_balance,balance_change
Index,,,,,,,,,,,,,,,,,
1,prime_auto,15000,8000,-5000,80,1,0.2,1.0,0,0,0,0,23000,10000,"(10000, 23000)",10000.0,-5000.0
2,subprime_auto,4000,2000,-1000,150,1,0.6,1.0,0,0,0,0,6000,3000,"(3000, 6000)",6000.0,2000.0
3,mtg_30_fixed,50000,30000,-15000,70,1,0.3,0.5,0,0,0,0,80000,35000,"(35000, 80000)",80000.0,30000.0
4,mtg_15_fixed,15000,8000,-3000,72,1,0.3,0.5,0,0,0,0,23000,12000,"(12000, 23000)",23000.0,8000.0
5,mtg_7_fixed,5000,2500,-2000,70,1,0.3,0.5,0,0,0,0,7500,3000,"(3000, 7500)",7500.0,2500.0


In [177]:
# Export updated results to Excel

df.to_excel('./results/results.xlsx')

#### Appendix

- Saving down some code from a previous iteration that's currently not used

In [360]:
# The below calculates the contribution of the GSIB score surcharge by line item

# Identify the GSIB-related columns
# gsib_cols = list(df.columns[-16:-4])

# # Multiply GSIB coefficients by starting balances and store in a new dataframe
# gsib_impact = df[gsib_cols].multiply(df['start'], axis='index')

# score_cols = []

# # Multiply the gsib balances by their respective scores
# for i, item in enumerate(gsib.index):
#     col_name = item + '_score'
#     score_cols.append(col_name)
#     gsib_impact[col_name] = gsib_impact.iloc[:,i] * float(gsib.iloc[i])

# # Sum across to calculate the "total GSIB score"
# gsib_impact['total_score'] = gsib_impact[score_cols].sum(axis=1)

# # Convert the score to the CET1 contribution, currently calibrated to be +3%
# gsib_impact['cet1_cont'] = gsib_impact['total_score'] * constraints.loc['GSIB', 'CET1']